<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/Copie_de_TP1_2022_colab_ETU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2022

date du document  :  22/09/2022

# TP1 Préparation de données


Restructuration de données et analyse de données.


## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/ens/bdle/TP1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Mounted at /content/drive


[]

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 67.5 MB/s 


Démarrer la session spark

In [ ]:
import os

# !find /usr/local/lib -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1664898638590


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  8


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas()


Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    for r in tabRequetes:
        if len(r.strip()) > 2:
          derniere = spark.sql(r)
    return display(derniere)

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

### URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL du dossier contenant les datasets ", PUBLIC_DATASET_URL)

URL du dossier contenant les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


### Données de mobilité

Données issues du dataset YFCC

In [ ]:
local_dir = "/local/data"
os.makedirs(local_dir, exist_ok=True)
os.listdir(local_dir)

[]

In [ ]:
from urllib import request

# download dataset if not already donwloaded
def download_file(web_dir, local_dir, file):
  local_file = local_dir + "/" + file
  web_file = web_dir + "/" + file
  if(os.path.isfile(local_file)):
    print(file, "is already stored")
  else:
    print("downloading from URL: ", web_file , "save in : " + local_file)
    request.urlretrieve(web_file , local_file)

# user visits
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15"
download_file(web_dir, local_dir, "userVisits-Toro.csv")

# poi 
web_dir = PUBLIC_DATASET + "YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15"
download_file(web_dir, local_dir, "POI-Toro.csv")



os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/userVisits-ijcai15/userVisits-Toro.csv save in : /local/data/userVisits-Toro.csv
downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=YFCC_POI_dataset_K_H_LIM/dataset_IJCAI_2015/data-ijcai15/poiList-ijcai15/POI-Toro.csv save in : /local/data/POI-Toro.csv


['POI-Toro.csv', 'userVisits-Toro.csv']

### Les visites

Lire le fichier des visites *sans* préciser le type des attributs.
Par défaut, tous les attributs sont considérés comme étant de type string.

In [ ]:
user_visits = spark.read.option("header", "True").option("delimiter", ";").format("csv").load(local_dir + "/" + "userVisits-Toro.csv")
user_visits.show(3)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID| dateTaken|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 3 rows

root
 |-- photoID: string (nullable = true)
 |-- userID: string (nullable = true)
 |-- dateTaken: string (nullable = true)
 |-- poiID: string (nullable = true)
 |-- poiTheme: string (nullable = true)
 |-- poiFreq: string (nullable = true)
 |-- seqID: string (nullable = true)



Lire le fichier en précisant le schéma : nom et type des attributs

In [ ]:
schema = "photoID long, userID String, date Long, poiID int, poiTheme String, poiFreq int, seqID int"

user_visits = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "userVisits-Toro.csv", schema = schema)
user_visits.persist()
user_visits.createOrReplaceTempView("user_visits")
user_visits.show(10)
user_visits.printSchema()

+----------+------------+----------+-----+---------+-------+-----+
|   photoID|      userID|      date|poiID| poiTheme|poiFreq|seqID|
+----------+------------+----------+-----+---------+-------+-----+
|7941504100|10007579@N00|1346844688|   30|Structure|   1538|    1|
|4886005532|10012675@N05|1142731848|    6| Cultural|    986|    2|
|4886006468|10012675@N05|1142732248|    6| Cultural|    986|    2|
|4885404441|10012675@N05|1142732373|    6| Cultural|    986|    2|
|4886008334|10012675@N05|1142732445|    6| Cultural|    986|    2|
|4886009150|10012675@N05|1142916492|    6| Cultural|    986|    3|
|7054481539|10012675@N05|1319327174|   13| Cultural|    964|    4|
|6908387594|10012675@N05|1319328255|   13| Cultural|    964|    4|
|6908381912|10012675@N05|1319331463|   13| Cultural|    964|    4|
|6908398496|10012675@N05|1319331886|   13| Cultural|    964|    4|
+----------+------------+----------+-----+---------+-------+-----+
only showing top 10 rows

root
 |-- photoID: long (nullable = 

Vérifier qu'il n'y a pas de date nulle

In [ ]:
%%sql
select count(1) as nb_dates_nulles
from user_visits
where date is null

-- autre solution avec "case"
--select count(case when isnull(date) then date end) as nb_date_nulles
--from user_visits


,nb_dates_nulles
0,0



Vérifier qu'il n'y a aucune séquence associée à plusieurs utilisateurs

In [ ]:
%%sql

select seqID
from user_visits
group by seqID
having count(distinct userID) > 1

,seqID



#### POI_sequence

**POI_sequence**(seqID, userID, poiID). La liste des POI d'une séquence.
Pour simplifier on compte chaque POI une seule fois par séquence et on ne considère pas l'ordre de visite des POI.

In [ ]:
%%sql
create or replace temp view POI_sequence as 
select distinct seqID, userID, poiID
from user_visits;

select * from POI_sequence

,seqID,userID,poiID
0,3,10012675@N05,6
1,4,10012675@N05,13
2,9,10014440@N06,24
3,14,10014440@N06,24
4,46,10316646@N07,3
...,...,...,...
95,619,18412989@N00,22
96,619,18412989@N00,16
97,620,18412989@N00,22
98,622,18412989@N00,16


### Les POI

In [ ]:
poi_schema = "poiID long, poiName String, latitude double, longitude double, theme String"

poi = spark.read.option("header", "True").option("delimiter", ";").csv(local_dir + "/" + "POI-Toro.csv", schema = poi_schema)
poi.show(3)
poi.printSchema()
poi.createOrReplaceTempView("POI")

+-----+------------------+--------+---------+-----+
|poiID|           poiName|latitude|longitude|theme|
+-----+------------------+--------+---------+-----+
|    1| Air_Canada_Centre|43.64333|-79.37917|Sport|
|    2|         BMO_Field|43.63278|-79.41861|Sport|
|    3|Maple_Leaf_Gardens|43.66222|-79.38028|Sport|
+-----+------------------+--------+---------+-----+
only showing top 3 rows

root
 |-- poiID: long (nullable = true)
 |-- poiName: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- theme: string (nullable = true)



In [ ]:
%%sql
cache table POI;

SELECT * 
From POI

,poiID,poiName,latitude,longitude,theme
0,1,Air_Canada_Centre,43.643330,-79.379170,Sport
1,2,BMO_Field,43.632780,-79.418610,Sport
2,3,Maple_Leaf_Gardens,43.662220,-79.380280,Sport
3,4,Rogers_Centre,43.641390,-79.389170,Sport
4,5,Woodbine_Racetrack,43.712525,-79.602042,Sport
5,6,Art_Gallery_of_Ontario,43.653890,-79.392780,Cultural
6,7,Hockey_Hall_of_Fame,43.646976,-79.377253,Cultural
7,8,Ripley%27s_Aquarium_of_Canada,43.642481,-79.386050,Cultural
8,9,Ontario_Science_Centre,43.716670,-79.338330,Cultural
9,10,Riverdale_Farm,43.667111,-79.361294,Cultural


la liste des thèmes

In [ ]:
%%sql
--create or replace temp view themes as
-- .............
--cache table themes;

--select * from themes

## Exercice 1

#### 1) Les 10 POI les plus visités

In [ ]:
%%sql

SELECT DISTINCT poiName, poiFreq 
from 
user_visits INNER JOIN POI ON user_visits.poiID==POI.poiID 
ORDER BY poiFreq DESC
Limit 10

,poiName,poiFreq
0,Royal_Ontario_Museum,4142
1,Nathan_Phillips_Square,3619
2,Yonge-Dundas_Square,3594
3,CN_Tower,3553
4,Air_Canada_Centre,3506
5,Rogers_Centre,3056
6,Hockey_Hall_of_Fame,2064
7,Toronto_Eaton_Centre,1874
8,Ripley%27s_Aquarium_of_Canada,1736
9,Kensington_Market,1701


#### 2) POI avec date
POI_date(...) Les POI avec la date détaillée composée des attributs : année, mois, jour, heure.

In [ ]:
%%sql

SELECT FROM_UNIXTIME(date,"y-m-d h'h'"), v.*
from user_visits v 


,"from_unixtime(date, y-m-d h'h')",photoID,userID,date,poiID,poiTheme,poiFreq,seqID
0,2012-31-5 11h,7941504100,10007579@N00,1346844688,30,Structure,1538,1
1,2006-30-19 1h,4886005532,10012675@N05,1142731848,6,Cultural,986,2
2,2006-37-19 1h,4886006468,10012675@N05,1142732248,6,Cultural,986,2
3,2006-39-19 1h,4885404441,10012675@N05,1142732373,6,Cultural,986,2
4,2006-40-19 1h,4886008334,10012675@N05,1142732445,6,Cultural,986,2
...,...,...,...,...,...,...,...,...
95,2008-53-9 8h,2654929774,10014440@N06,1215593613,25,Shopping,1701,10
96,2008-53-9 8h,2654104231,10014440@N06,1215593634,25,Shopping,1701,10
97,2008-54-9 8h,2654930912,10014440@N06,1215593650,25,Shopping,1701,10
98,2008-54-9 8h,2654105249,10014440@N06,1215593655,25,Shopping,1701,10


#### 3) Le nombre de POI par utilisateur

In [ ]:
%%sql 

select  userID, count(distinct poiID)
from user_visits
GROUP BY userID

,userID,count(DISTINCT poiID)
0,11090433@N05,12
1,12641532@N00,2
2,16693950@N00,11
3,19761391@N06,6
4,20605823@N00,2
...,...,...
95,27008470@N00,3
96,33739230@N02,2
97,51260381@N05,6
98,53400644@N05,2


#### 4) Trajectoire
Trajectoire(userID, seqID, listPOI)
Pour chaque séquence, la liste ORDONNEE des POI visités

In [ ]:
from pyspark.sql.functions import udf

def order_by_poi_by_date(c):
  c=sorted(c,key=lambda x:x[1])
  l=[]
  for i in c:
    if i[0] not in l :
      l.append(i[0])
  return l

f1ud=udf(lambda x:order_by_poi_by_date(x), ArrayType(IntegerType()))

user_visits.groupBy("userID","seqID").\
agg(collect_list(array("poiID","date")).\
    alias("poi and date")).\
    select("userID","seqID", f1ud(col("poi and date")).alias("poiID_visited")).\
    createOrReplaceTempView("H")

In [ ]:
%%sql

select * from H

,userID,seqID,poiID_visited
0,10007579@N00,1,[30]
1,10012675@N05,2,[6]
2,10012675@N05,3,[6]
3,10012675@N05,4,[13]
4,10014440@N06,5,[24]
...,...,...,...
95,10627620@N06,96,[21]
96,10627620@N06,97,[8]
97,10627620@N06,98,[3]
98,10627620@N06,99,"[3, 23, 27]"


#### 5) Transitions
Transitions(poi1, poi2, transitions) avec nb étant le nombre de déplacements directs de poi1 à poi2

In [ ]:
from pyspark.sql.types import StructType

def transition_poi(x):
  return [(x[i],x[i+1]) for i in range(0,len(x)-1)]

#ftr1=udf(lambda x:transition_poi(x), ArrayType(StructType([Struct("poi1",IntegerType()),StructField("poi2",IntegerType())])))
ftr1=udf(lambda x:transition_poi(x), ArrayType(StructType(
    [
        StructField("poi1",IntegerType(),False),\
        StructField("poi2",IntegerType(),False)
    ]
    )
)
)
spark.udf.register("transition_poi",ftr1)

<function __main__.<lambda>(x)>

In [ ]:
%%sql

select userID, seqID, explode(transition_poi(poiID_visited)) as start_end
from H
where size(transition_poi(poiID_visited))>1

,userID,seqID,start_end
0,10502709@N05,58,"(7, 11)"
1,10502709@N05,58,"(11, 27)"
2,10502709@N05,67,"(28, 23)"
3,10502709@N05,67,"(23, 22)"
4,10502709@N05,71,"(22, 28)"
...,...,...,...
95,18948547@N00,634,"(28, 23)"
96,20075574@N00,675,"(22, 6)"
97,20075574@N00,675,"(6, 25)"
98,20342758@N00,681,"(23, 28)"


In [ ]:
%%sql

select start_end,count(*) as nb_transition from
(
select explode(transition_poi(poiID_visited)) as start_end
from H
where size(transition_poi(poiID_visited))>1
)
group by start_end



,start_end,nb_transition
0,"(11, 27)",3
1,"(28, 23)",26
2,"(4, 16)",12
3,"(4, 8)",6
4,"(28, 16)",4
...,...,...
95,"(11, 24)",2
96,"(22, 27)",2
97,"(25, 19)",1
98,"(21, 16)",2


#### 6) Distances entre POIs
Distance(poi1, poi2, distance)

In [ ]:
#a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2

In [ ]:
%%sql


select p1.poiName,p2.poiName, pow(sin((p1.latitude-p2.latitude)/2),2)+cos(p1.latitude)*cos(p2.latitude)*pow(sin((p1.longitude-p2.longitude)/2),2)
from POI p1,POI p2
where not p1.poiID=p2.poiID

,poiName,poiName,"(pow(SIN(((latitude - latitude) / 2)), 2) + ((COS(latitude) * COS(latitude)) * pow(SIN(((longitude - longitude) / 2)), 2)))"
0,Air_Canada_Centre,BMO_Field,0.000372
1,Air_Canada_Centre,Maple_Leaf_Gardens,0.000089
2,Air_Canada_Centre,Rogers_Centre,0.000023
3,Air_Canada_Centre,Woodbine_Racetrack,0.012438
4,Air_Canada_Centre,Art_Gallery_of_Ontario,0.000069
...,...,...,...
95,Rogers_Centre,Riverdale_Farm,0.000339
96,Rogers_Centre,Royal_Ontario_Museum,0.000176
97,Rogers_Centre,Toronto_Zoo,0.017849
98,Rogers_Centre,Casa_Loma,0.000430


#### 7) DuréeVisitePOI
La durée moyenne de visite d'un POI.

In [ ]:
%%sql 
with user_visits_time_poi as(
select  v1.poiID as poiID, max(v2.date)-min(v1.date) as time
from user_visits v1, user_visits v2
where  
v1.userID=v2.userID and
v1.poiID=v2.poiID and 
v1.seqID=v2.seqID
group by v1.userID , v1.poiID)



select poiID,avg(time) 
from user_visits_time_poi
group by poiID

,poiID,avg(time)
0,14,2.647538e+05
1,13,1.063678e+07
2,12,1.634405e+07
3,18,1.037400e+03
4,6,1.286278e+07
5,16,1.411887e+07
6,23,2.229325e+07
7,9,8.010834e+06
8,17,3.430786e+07
9,10,1.615131e+07


## Exercice 2

Charger les données de Geonames

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile

#geonames
web_dir = PUBLIC_DATASET + "geonames_ALL"

download_file(web_dir, local_dir, "allCountries.zip")

#unzip
local_file = local_dir + "/" + "allCountries.zip"
if(os.path.isfile(local_file)):
    print(local_file, "is already stored")
else:
  with zipfile.ZipFile(local_file, 'r') as zip_ref:
    zip_ref.extractall(local_dir)

os.listdir(local_dir)

downloading from URL:  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4/download?path=geonames_ALL/allCountries.zip save in : /local/data/allCountries.zip
/local/data/allCountries.zip is already stored


['POI-Toro.csv', 'userVisits-Toro.csv', 'allCountries.zip']

In [ ]:
with zipfile.ZipFile("/local/data/allCountries.zip", 'r') as zip_ref:
  zip_ref.extractall(local_dir)

In [ ]:
geonames =  spark.read.option("header", "False").option("delimiter", "\t").format("csv").load(local_dir + "/" + "allCountries.txt")

geonames.show(3)


+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|    _c0|                 _c1|                 _c2|                 _c3|     _c4|    _c5|_c6|_c7|_c8|  _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|          _c17|      _c18|
+-------+--------------------+--------------------+--------------------+--------+-------+---+---+---+-----+----+----+----+----+----+----+----+--------------+----------+
|2986043|  Pic de Font Blanca|  Pic de Font Blanca|Pic de Font Blanc...|42.64991|1.53335|  T| PK| AD| null|  00|null|null|null|   0|null|2860|Europe/Andorra|2014-11-05|
|2994701|            Roc Mélé|            Roc Mele|Roc Mele,Roc Mele...|42.58765|1.74028|  T| MT| AD|AD,FR|  00|null|null|null|   0|null|2803|Europe/Andorra|2014-11-05|
|3007683|Pic des Langounelles|Pic des Langounelles|Pic des Langounelles|42.61203|1.47364|  T| PK| AD|AD,FR|  00|null|null|null|   0|null|2685|Europe/Andorr

#### 1) Geonames
Définir le schéma, limité au 9 premiers attributs, d'après le document [readme.txt](https://nuage.lip6.fr/s/kaBX3sF3YEzMZHK) dans le dossier [geonames_ALL](https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4?path=%2Fgeonames_ALL)

In [ ]:
for i in range(9,19):
  s="_c"+str(i)
  geonames=geonames.drop(s)

In [ ]:
schema = StructType([
    StructField("geoname", StringType(),False),
    StructField("name", StringType(),False),
    StructField("ascii_name", StringType(),False),
    StructField("alternatename", StringType(),False),
    StructField("latitude", StringType(),False),
    StructField("longitude", StringType(),False),
    StructField("feature class", StringType(),False),
    StructField("feature code", StringType(),False),
    StructField("country code", StringType(),False)
])

In [ ]:
geoname=spark.createDataFrame(geonames.collect(), schema=schema)

In [ ]:
geoname=geoname.withColumn("latitude",col("latitude").cast(DoubleType()))
geoname=geoname.withColumn("longitude",col("longitude").cast(DoubleType()))
geoname=geoname.withColumn("geoname",col("geoname").cast(IntegerType()))

In [ ]:
%%sql


select * from geoname

#### 2) Association entre les POI et Geonames
Compléter les POI avec des attributs de geonames (par exemple name et feature code)

In [ ]:
geonames=geonames.withColumn("_c4",col("_c4").cast(DoubleType()))
geonames=geonames.withColumn("_c5",col("_c5").cast(DoubleType()))

In [ ]:
poi2=poi.join(geonames,(geonames._c4==poi.latitude) & (geonames._c5==poi.longitude), "inner")
poi2.createOrReplaceTempView("POI2")

In [ ]:
%%sql


select * from POI2

,poiID,poiName,latitude,longitude,theme,_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8
0,14,Canadian_National_Exhibition,43.63333,-79.41667,Amusement,6301425,Toronto Island,Toronto Island,CWTZ,43.63333,-79.41667,S,STNM,CA
